In [430]:
import numpy as np
import pandas as pd

In [431]:
with pd.ExcelFile('input/PD Bechdel Test.xlsx') as xl:
    df_web = pd.read_excel(xl, sheet_name='Webscraping')
    df_html = pd.read_excel(xl, sheet_name='html')

In [432]:
pd.set_option("display.max_colwidth", 1000)
df_web.head()

,DownloadData,Year
0,"<a href=""http://us.imdb.com/title/tt3155794/""><img src=""/static/nopass.png"" alt=""[[0]]"" title=""[Fewer than two women in this movie]""></a>\n\t\t<a id=""movie-9602"" href=""/view/9602/passage_de_venus/"">Passage de Venus</a> <a href=""/view/9602/passage_de_venus/"" onclick=""showComments('9602'); return false;""><img src=""/static/comments.png"" alt=""[1 comment(s) available]""\n\t\t\t\tid=""comment-img-9602"" style=""height: 10px; width: 10px;""\n title=""1 comment""></a> </div> <div style=""border: 2px dashed white; margin-top: 3px; margin-right: 14px; padding: 3px; background-color: #CFE8CC; text-align: center;""> <div style=""margin-top: 7px; font-size: 80%;""><a href=""/search/"">You can also search or list per year</a>, or <a href=""?list=all"">list all movies</a>.</div>\n</div>\n</div>\n\n<div>\n<h2>About</h2>\n<p>The <b>Bechdel Test</b>, or <b><a href=""https://www.independent.co.uk/arts-entertainment/films/news/please-stop-calling-it-the-bechdel-test-says-alison-bechdel-10474730.htm...",1874
1,"<a href=""http://us.imdb.com/title/tt14495706/""><img src=""/static/nopass.png"" alt=""[[0]]"" title=""[Fewer than two women in this movie]""></a>\n\t\t<a id=""movie-9804"" href=""/view/9804/la_rosace_magique/"">La Rosace Magique</a> <a href=""/view/9804/la_rosace_magique/"" onclick=""showComments('9804'); return false;""><img src=""/static/comments.png"" alt=""[1 comment(s) available]""\n\t\t\t\tid=""comment-img-9804"" style=""height: 10px; width: 10px;""\n title=""1 comment""></a> </div>",1877
2,"<a href=""http://us.imdb.com/title/tt12592084/""><img src=""/static/nopass.png"" alt=""[[0]]"" title=""[Fewer than two women in this movie]""></a>\n\t\t<a id=""movie-9806"" href=""/view/9806/le_singe_musicien/"">Le singe musicien</a> <a href=""/view/9806/le_singe_musicien/"" onclick=""showComments('9806'); return false;""><img src=""/static/comments.png"" alt=""[1 comment(s) available]""\n\t\t\t\tid=""comment-img-9806"" style=""height: 10px; width: 10px;""\n title=""1 comment""></a> </div>",1878
3,"<a href=""http://us.imdb.com/title/tt2221420/""><img src=""/static/nopass.png"" alt=""[[0]]"" title=""[Fewer than two women in this movie]""></a>\n\t\t<a id=""movie-9603"" href=""/view/9603/sallie_gardner_at_a_gallop/"">Sallie Gardner at a Gallop</a> <a href=""/view/9603/sallie_gardner_at_a_gallop/"" onclick=""showComments('9603'); return false;""><img src=""/static/comments.png"" alt=""[1 comment(s) available]""\n\t\t\t\tid=""comment-img-9603"" style=""height: 10px; width: 10px;""\n title=""1 comment""></a> </div>",1878
4,"<a href=""http://us.imdb.com/title/tt7816420/""><img src=""/static/nopass.png"" alt=""[[0]]"" title=""[Fewer than two women in this movie]""></a>\n\t\t<a id=""movie-9816"" href=""/view/9816/athlete_swinging_a_pick/"">Athlete Swinging a Pick</a> <a href=""/view/9816/athlete_swinging_a_pick/"" onclick=""showComments('9816'); return false;""><img src=""/static/comments.png"" alt=""[1 comment(s) available]""\n\t\t\t\tid=""comment-img-9816"" style=""height: 10px; width: 10px;""\n title=""1 comment""></a> </div>",1881


In [433]:
df_html.head()

,Char,Numeric,Named,Description
0,NaN,code,code,NaN
1,NaN,&#32;,NaN,space
2,!,&#33;,NaN,exclamation mark
3,"""",&#34;,&quot;,double quote
4,#,&#35;,NaN,number


In [434]:
df_html.iat[1,0] = ' '
df_html = df_html.iloc[1:]
html_num_to_char = dict(zip(df_html['Numeric'], df_html['Char']))

In [435]:
df_html = df_html.dropna(subset=['Named'])
html_named_to_char = dict(zip(df_html['Named'], df_html['Char']))

In [436]:
html_amp_to_and = {'&amp;':'&'}

In [437]:
for k,v in html_amp_to_and.items():
    df_web['DownloadData'] = df_web['DownloadData'].apply(lambda df_x : df_x.replace(repr(k), repr(v)))

for k,v in html_num_to_char.items():
    df_web['DownloadData'] = df_web['DownloadData'].apply(lambda df_x : df_x.replace(repr(k), repr(v)))
    
for k,v in html_named_to_char.items():
    df_web['DownloadData'] = df_web['DownloadData'].apply(lambda df_x : df_x.replace(repr(k), repr(v)))

In [438]:
df_web.iat[100,0]

'<a href="http://us.imdb.com/title/tt0258104/"><img src="/static/nopass.png" alt="[[0]]" title="[Fewer than two women in this movie]"></a>\n\t\t<a id="movie-5437" href="/view/5437/reve_et_realite/">Reve et realite</a>   </div>'

In [439]:
# str.replaceを続けてくっつけると、エラーを吐き出す
df_web['Categorisation'] = df_web['DownloadData'].str.extract('title="(.+)]"></a', expand=True)
df_web['Categorisation'] = df_web['Categorisation'].str.replace('[', '')

In [440]:
df_web['Movie'] = df_web['DownloadData'].str.extract('href="/view/(.+)</a>', expand=True)
df_web['Movie'] = df_web['Movie'].str.split('">', expand=True)[1]

In [441]:
df_web['Pass/Fail'] = df_web["DownloadData"].apply(lambda df_x: 'Fail' if 'nopass.png' in df_x else 'Pass')

In [442]:
df_web['Categorisation'].drop_duplicates()

0                                                                                            Fewer than two women in this movie
64                                                 There are two or more women in this movie, but they don't talk to each other
86                       There are two or more women in this movie and they talk to each other about something other than a man
128                                     There are two or more women in this movie, but they only talk to each other about a man
150    There are two or more women in this movie and they talk to each other about something other than a man, although dubious
Name: Categorisation, dtype: object

In [443]:
Categori_rank = {"Fewer than two women in this movie": 5,
"There are two or more women in this movie, but they don't talk to each other": 4,
"There are two or more women in this movie and they talk to each other about something other than a man": 1,
"There are two or more women in this movie, but they only talk to each other about a man": 3,
"There are two or more women in this movie and they talk to each other about something other than a man, although dubious": 2}

df_web['Ranking'] = df_web['Categorisation'].map(Categori_rank)

In [444]:
col = ['Movie', 'Year', 'Pass/Fail', 'Ranking', 'Categorisation']
df_web.to_csv('output/output-2022', columns=col, index=False)

In [445]:
output = pd.read_csv('output/output-2022')
output.head()

,Movie,Year,Pass/Fail,Ranking,Categorisation
0,Passage de Venus,1874,Fail,5,Fewer than two women in this movie
1,La Rosace Magique,1877,Fail,5,Fewer than two women in this movie
2,Le singe musicien,1878,Fail,5,Fewer than two women in this movie
3,Sallie Gardner at a Gallop,1878,Fail,5,Fewer than two women in this movie
4,Athlete Swinging a Pick,1881,Fail,5,Fewer than two women in this movie


In [447]:
output.groupby(by=['Year', 'Ranking'])['Movie'].count()

Year  Ranking
1874  5           1
1877  5           1
1878  5           2
1881  5           1
1883  5           1
                 ..
2021  2          12
      3          13
      4          12
      5          10
2022  1           1
Name: Movie, Length: 528, dtype: int64